In [1]:
import resource
def usage():
    res = resource.getrusage(resource.RUSAGE_SELF)
    return {
        "usr": res[0],
        "sys": res[1],
        "time": res[0] + res[1], # sum of user and system mode
        "mem": res[2]/1024.0
    }

In [2]:
before = usage()

In [3]:
import pandas as pd
import numpy as np

In [4]:
df1 = pd.read_stata('http://www.stata-press.com/data/r11/auto.dta')
#df1

In [5]:
grouped_by_rep78 = df1.groupby('rep78')

# Table 1

In [6]:
grouped_by_rep78.agg(
    **{
        'Freq': pd.NamedAgg(column='rep78', aggfunc='count'),
    }
)

,Freq
rep78,
1.0,2
2.0,8
3.0,30
4.0,18
5.0,11


# Table 2

In [7]:
grouped_by_rep78.agg(
    **{
        'N(mpg)': pd.NamedAgg(column='mpg', aggfunc='count'),
        'mean(mpg)': pd.NamedAgg(column='mpg', aggfunc='mean'),
        'med(mpg)': pd.NamedAgg(column='mpg', aggfunc='median'),
        'max(price)': pd.NamedAgg(column='price', aggfunc='max'),
        'min(length)': pd.NamedAgg(column='length', aggfunc='min'),
    }
)

,N(mpg),mean(mpg),med(mpg),max(price),min(length)
rep78,,,,,
1.0,2,21.000000,21.0,4934,180
2.0,8,19.125000,18.0,14500,179
3.0,30,19.433333,19.0,15906,142
4.0,18,21.666667,22.5,9735,147
5.0,11,27.363636,30.0,11995,155


# Table 3

In [8]:
t3 = grouped_by_rep78.agg(
    **{
        'Domestic': pd.NamedAgg(column='foreign', aggfunc=lambda x: x.eq("Domestic").sum()),
        'Foreign': pd.NamedAgg(column='foreign', aggfunc=lambda x: x.eq("Foreign").sum()),
    }
)
t3['Total'] = t3.iloc[:, 0:2].sum(axis=1)
t3.loc['Total'] = t3.sum(numeric_only=True, axis=0)
t3.append(t3.sum(numeric_only=True), ignore_index=True)
t3

,Domestic,Foreign,Total
rep78,,,
1.0,2,0,2
2.0,8,0,8
3.0,27,3,30
4.0,9,9,18
5.0,2,9,11
Total,48,21,69


# Table 4

In [9]:
def f(x):
    d = {}
    d['Domestic'] = x[x['foreign'] == 'Domestic']['mpg'].mean()
    d['Foreign'] = x[x['foreign'] == 'Foreign']['mpg'].mean()
    
    d['Total'] = x['mpg'].mean()

    return pd.Series(d, index=['Domestic', 'Foreign', 'Total'])

t4 = grouped_by_rep78.apply(f)

t4.loc['Total'] = df1.groupby('foreign')['mpg'].mean()
t4

,Domestic,Foreign,Total
rep78,,,
1.0,21.000000,NaN,21.000000
2.0,19.125000,NaN,19.125000
3.0,19.000000,23.333333,19.433333
4.0,18.444444,24.888889,21.666667
5.0,32.000000,26.333333,27.363636
Total,19.826923,24.772727,NaN


In [10]:
after = usage()

print("time elapsed = {}".format(after["time"] - before["time"]))
print("usr elapsed = {}".format(after["usr"] - before["usr"]))
print("sys elapsed = {}".format(after["sys"] - before["sys"]))
print("memory used = {}".format(after["mem"] - before["mem"]))

time elapsed = 1.6096859999999997
usr elapsed = 0.996708
sys elapsed = 0.612978
memory used = 48.69921875


In [11]:
import sys
print("dataframe size in memory {} kB".format(sys.getsizeof(df1) / 1024.0))

dataframe size in memory 10.08984375 kB
